# Get review data and teams keywords

In [4]:
import pandas as pd
import os
from tqdm import tqdm
import json

In [22]:
REVIEW_PATH = '../Resources/1_clean_review_data'

KEYWORDS_PATH = '../Resources/2_keywords_for_teams_data'

DATA_PATH = '../Resources'

REVIEW_COL = 'Review'
TITLE_COL = 'Title'

In [3]:
data = pd.read_csv(os.path.join(REVIEW_PATH,'cleanAppStore.csv'))

In [7]:
with open(os.path.join(KEYWORDS_PATH,'team_keywords.json'),'r') as f:
    team_names_keywords = json.load(f)

with open(os.path.join(KEYWORDS_PATH,'grouped_team_keywords.json'),'r') as f:
    groups_keywords = json.load(f)
    
with open(os.path.join(KEYWORDS_PATH,'grouped_team_mapping.json'),'r') as f:
    mapping = json.load(f)

# Extract keywords

In [11]:
from rutermextract import TermExtractor
term_extractor = TermExtractor()

## No weights, i.e. numbers of occurances

In [36]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [78]:
reviews_keywords = {}
empty_index = []
for i in tqdm(data.index):
    reviews_keywords[i] = []
    for term in term_extractor(data.iloc[i][REVIEW_COL]):
        reviews_keywords[i].append(term.normalized)
    for term in term_extractor(data.iloc[i][TITLE_COL]):
        reviews_keywords[i].append(term.normalized)
    reviews_keywords[i] = list(set(reviews_keywords[i]))
    if len(reviews_keywords[i])==0:
        del reviews_keywords[i]
        empty_index.append(i)

100%|██████████| 11584/11584 [00:51<00:00, 225.74it/s]


In [17]:
data.iloc[0][REVIEW_COL]

'Не скачивается стала обновлять и зависло приложение что случилось'

In [42]:
data.iloc[0][TITLE_COL]

'Сбербанк онлайн'

In [41]:
reviews_keywords

{0: ['сбербанк', 'приложение'],
 1: ['обновление', 'неделя', 'мои друзья', 'приложение'],
 2: ['функция',
  'работа',
  'последнее обновление',
  'обновлённая версия',
  'приложение',
  'сканирование лица',
  'добрый день'],
 3: ['подключение wi-fi'],
 4: ['минус', 'applepay', 'баланс', 'mastercard'],
 5: ['перевод',
  'нея',
  'номер телефона',
  'деньга',
  'поддержка',
  'банка',
  'аккаунт',
  'карта',
  '2 карты',
  'проблема',
  'такая низкая оценка',
  'такой крупный банка',
  'приложение',
  'ошибка',
  'тупой бот',
  'нонсенс'],
 6: ['история', 'возможность'],
 7: ['2020 год',
  'вид',
  'ваши жёлтые конкуренты',
  'банка',
  'сии поры',
  'доставка',
  'отдетение',
  'ваше банка',
  'огромная база зарплатников',
  'все счёт',
  'банк',
  'сбер',
  'одно город',
  'другие банка',
  'ребята',
  'офис',
  'пенсионеры',
  'переводы',
  'карта',
  'закрой',
  'другое отделение'],
 8: ['перевод денег', 'банка'],
 9: ['дипломированные мошенники'],
 10: ['половина операций',
  'прило

In [50]:
from gensim.summarization import keywords
import re

In [66]:
reviews_keywords_gensim = {}
for i in tqdm(data.index):
    reviews_keywords_gensim[i] = []
    text = data.iloc[i][REVIEW_COL]
    lemm = []
    for word in text.split(u' '):
        if re.match(u'([^a-zа-яe ̈]+)', word):
            word = morph.parse(word)[0].normal_form
        lemm.append(word)
    text = ' '.join(lemm)
    reviews_keywords_gensim[i]+=keywords(text).split('\n')
    text = data.iloc[i][TITLE_COL]
    lemm = []
    for word in text.split(u' '):
        if re.match(u'([^a-zа-яe ̈]+)', word):
            word = morph.parse(word)[0].normal_form
            lemm.append(word)
    text = ' '.join(lemm)
    reviews_keywords_gensim[i]+=keywords(text).split('\n')
#     reviews_keywords_gensim[i] = list(set(reviews_keywords_gensim[i]))

100%|██████████| 11584/11584 [00:28<00:00, 412.15it/s]


In [67]:
reviews_keywords_gensim

{0: ['стала', ''],
 1: ['пытаюсь', ''],
 2: ['функция', 'день', ''],
 3: ['требует', ''],
 4: ['оплатить', ''],
 5: ['деньги',
  'связаться',
  'что',
  'крупном банке',
  'приоритетная',
  'как',
  'проблему',
  ''],
 6: ['дико', ''],
 7: ['карту',
  'внутри банка',
  'базе',
  'банки',
  'доставка',
  'переводы',
  'тем',
  'вид',
  'ребят',
  ''],
 8: ['внутри', ''],
 9: ['', ''],
 10: ['раньше', 'время', 'показывает', ''],
 11: ['супер', ''],
 12: ['', ''],
 13: ['как', 'без', ''],
 14: ['удобное', ''],
 15: ['время', 'интерфеисе', ''],
 16: ['обновить', ''],
 17: ['остаток', 'бюджет', ''],
 18: ['последнего', ''],
 19: ['грузится', ''],
 20: ['', ''],
 21: ['один', 'само', ''],
 22: ['переводить', 'как', 'только', 'тыс', 'сосать', 'бесчеловечныи', ''],
 23: ['телефоне', 'работает', ''],
 24: ['этим', 'ежемесячнои', 'своего', 'банковское', 'повторить', ''],
 25: ['выводить', 'где', 'кредитнои', ''],
 26: ['история', 'тоже', 'версия', ''],
 27: ['', ''],
 28: ['версия',
  'для планш

# Conclusion: Gensim worse for keywords extracting than library

# Common words baseline

In [107]:
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
from numpy import argmax
import sklearn.preprocessing
import numpy as np

In [79]:
review_keywords_transformed, groups_keywords_transformed = transform(reviews_keywords, groups_keywords)

  0%|          | 11/11139 [00:00<01:46, 104.53it/s]


All Tokens:
65731


100%|██████████| 26/26 [00:00<00:00, 107.68it/s]


In [100]:
def transform(review_keywords, groups_keywords):
    tokens = []
    for key in review_keywords:
        tokens+=review_keywords[key]
    for key in groups_keywords:
        tokens+=groups_keywords[key]
    print()
    print('All Tokens:')
    print(len(tokens))

    results = []
    label_enc = sklearn.preprocessing.LabelEncoder()
    onehot_enc = sklearn.preprocessing.OneHotEncoder()
    
    encoded_all_tokens = label_enc.fit_transform(list(set(tokens)))
    encoded_all_tokens = encoded_all_tokens.reshape(len(encoded_all_tokens), 1)
    
    onehot_enc.fit(encoded_all_tokens)
    
    review_keywords_transformed = {}
    for key in tqdm(review_keywords):
        
        encoded_words = label_enc.transform(review_keywords[key])
        
        
        encoded_words = onehot_enc.transform(encoded_words.reshape(len(encoded_words), 1))
        

        review_keywords_transformed[key]=(np.mean(encoded_words.toarray(), axis=0))
        
    groups_keywords_transformed = {}
    for key in tqdm(groups_keywords):
        
        encoded_words = label_enc.transform(groups_keywords[key])
        
        
        encoded_words = onehot_enc.transform(encoded_words.reshape(len(encoded_words), 1))
        

        groups_keywords_transformed[key]=(np.mean(encoded_words.toarray(), axis=0))
    
    return review_keywords_transformed, groups_keywords_transformed

In [101]:
predictions = {}

for key in tqdm(review_keywords_transformed):
    predictions[key] = {}
    for group in groups_keywords_transformed:
        score = euclidean_distances([groups_keywords_transformed[group]], [review_keywords_transformed[key]])[0][0]
        predictions[key][group] = score

100%|██████████| 11139/11139 [00:52<00:00, 211.49it/s]


In [105]:
import random
from collections import Counter,OrderedDict

#sampling with replacement
sampling = random.choices(list(reviews_keywords.keys()), k=20)

### Eucleadean distance

In [118]:
for sample in sampling:
    print('---')
    print('Review')
    print(data.iloc[sample][TITLE_COL], data.iloc[sample][REVIEW_COL])
    print()
    scores = OrderedDict(sorted(predictions[sample].items(), key=lambda kv: kv[1]))
    print('Nearest team')
    print(list(scores.keys())[0], scores[list(scores.keys())[0]])
    print('---')

---
Review
Стало очень громоздким Отличное приложение, прекрасно выполняет свои функции, но с годами становится все более и более громоздким. Последнее обновление не несло ничего по сути кроме украшений а весило больше 250мб. Лично у меня на телефоне места всегда не хватает.. Эти открытки и сторис мне лично не нужны, я была бы счастлива отключить и скрыть их и тем освободить немного места на телефоне, пока что удалось в настройках скрыть часть не нужного мне контента на главном экране приложения, но на занимаемый объем памяти это никак не повлияло.. Сейчас это одно из самых громоздких приложений, но оно необходимо так что приходится терпеть.
Была бы безмерно благодарна за лайт-версию без всяких поющих отрыток, стикеров, сторисов, и тому подобного а только с базовыми функциями или за возможность отключить и скрыть их в текущей версии освободив тем немного места. Думаю я не одна такая..

Nearest team
Госуслуги 5.0
---
---
Review
Третий день не работает ! Третий день не работает ! Возможн

### Conclusion:
Not weighted eucleadean distance predict best scores for groups with the smallest number of keywords (Госуслуги)

In [109]:
predictions_cos = {}

for key in tqdm(review_keywords_transformed):
    predictions_cos[key] = {}
    for group in groups_keywords_transformed:
        score = cosine_similarity([groups_keywords_transformed[group]], [review_keywords_transformed[key]])[0][0]
        predictions_cos[key][group] = score

100%|██████████| 11139/11139 [01:30<00:00, 122.74it/s]


In [119]:
for sample in sampling:
    print('---')
    print('Review')
    print(data.iloc[sample][TITLE_COL], data.iloc[sample][REVIEW_COL])
    print()
    scores = OrderedDict(sorted(predictions_cos[sample].items(), key=lambda kv: kv[1], reverse = True))
    print('Nearest team')
    print(list(scores.keys())[0], scores[list(scores.keys())[0]])
    print('---')

---
Review
Стало очень громоздким Отличное приложение, прекрасно выполняет свои функции, но с годами становится все более и более громоздким. Последнее обновление не несло ничего по сути кроме украшений а весило больше 250мб. Лично у меня на телефоне места всегда не хватает.. Эти открытки и сторис мне лично не нужны, я была бы счастлива отключить и скрыть их и тем освободить немного места на телефоне, пока что удалось в настройках скрыть часть не нужного мне контента на главном экране приложения, но на занимаемый объем памяти это никак не повлияло.. Сейчас это одно из самых громоздких приложений, но оно необходимо так что приходится терпеть.
Была бы безмерно благодарна за лайт-версию без всяких поющих отрыток, стикеров, сторисов, и тому подобного а только с базовыми функциями или за возможность отключить и скрыть их в текущей версии освободив тем немного места. Думаю я не одна такая..

Nearest team
интерфейс приложения 0.0
---
---
Review
Третий день не работает ! Третий день не работае

### Conclusion:
Cosine distance predict scores as zero as number of keywords in groups is small, but predicts "карты" not bad

# Cosine distance for teams

In [121]:
review_keywords_transformed, teams_keywords_transformed = transform(reviews_keywords, team_names_keywords)

  0%|          | 10/11139 [00:00<01:55, 96.44it/s]


All Tokens:
65764


100%|██████████| 42/42 [00:00<00:00, 98.47it/s] 


In [129]:
predictions_cos_teams = {}

for key in tqdm(review_keywords_transformed):
    predictions_cos_teams[key] = {}
    for team in teams_keywords_transformed:
        score = cosine_similarity([teams_keywords_transformed[team]], [review_keywords_transformed[key]])[0][0]
        predictions_cos_teams[key][team] = score

100%|██████████| 11139/11139 [02:24<00:00, 77.10it/s]


In [130]:
for sample in sampling:
    scores = OrderedDict(sorted(predictions_cos_teams[sample].items(), key=lambda kv: kv[1], reverse = True))
    if scores[list(scores.keys())[0]]!=0:
        print('---')
        print('Review')
        print(data.iloc[sample][TITLE_COL], data.iloc[sample][REVIEW_COL])
        print()
        print('Nearest team')
        print(list(scores.keys())[0], scores[list(scores.keys())[0]])
        print('---')

---
Review
Баг Внизу на вкладке «диалоги» постоянно висит уведомление о непрочитанных диалогах, хотя их там нет. Когда открываешь вкладку, уведомление исчезает, после повторного входа в приложение появляется снова

Nearest team
Sberbank ID B2C 0.16903085094570333
---
---
Review
Куда делись раздел расход по счету Пропала в начале страницы расход по карте ??????

Nearest team
Плановый и досрочный перевыпуск дебетовых карт 0.13608276348795434
---
---
Review
Загрузка Сделайте приложение меньшим размером. Требуется подключение к WiFi, так как файл весит более 200 мб.

Nearest team
Global Navigation 0.07412493166611012
---
---
Review
Нет отображения истории платежей.(((( Спасибо, исправили.

Nearest team
iOS Platform 0.10783277320343841
---
---
Review
Проблема есть Не работает оплата по штрих-коду и QR-коду! Не найден лицевой счет!

Nearest team
ЕФС.Платежи МП 0.08944271909999157
---
---
Review
Выбор карты для платежей «по умолчанию» Буквально сегодня обнаружили, что сменилась карта по умолч